**Download the datasets**

In [1]:
import yfinance as yf # 資料來源： Yahoo Finance
import numpy as np

# Define USD-based currency pairs to download (Yahoo Finance ticker format)
symbols = ['USDJPY=X', 'EURUSD=X', 'GBPUSD=X']

# Download historical FX data
fx_data = {}
for symbol in symbols:
    data = yf.download(symbol, period="12y", interval="1d")
    fx_data[symbol] = data['Close']
    print(f"{symbol} - {len(data)} rows downloaded.")

# Convert to pure numpy arrays
fx_numpy_list = [fx_data[s].to_numpy() for s in symbols]

# Align all currency pairs to the same length
min_len = min(len(arr) for arr in fx_numpy_list)
fx_numpy_list = [arr[-min_len:] for arr in fx_numpy_list]

# Combine into shape (3, N) → each currency pair is one row (dimension)
fx_combined = np.stack(fx_numpy_list, axis=0)   # fx_combined shape: (3, 3125, 1)
fx_data = fx_combined.reshape(3, -1)            # fx_data shape: (3, 3125)

C:\Users\HP\AppData\Local\Temp\ipykernel_15608\3986527490.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="12y", interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_15608\3986527490.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="12y", interval="1d")


USDJPY=X - 3125 rows downloaded.


[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_15608\3986527490.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="12y", interval="1d")


EURUSD=X - 3125 rows downloaded.


[*********************100%***********************]  1 of 1 completed

GBPUSD=X - 3125 rows downloaded.


In [2]:
import pandas as pd

# Transpose → shape becomes (N, 3), suitable for DataFrame format
fx_data = fx_data.T  # Example: shape (N, 3) # N = number of days

# Define currency pair column names (without "=X" for cleaner column names)
symbols = ['USDJPY', 'EURUSD', 'GBPUSD']

# Convert to DataFrame
df = pd.DataFrame(fx_data, columns=symbols) # shape: (3125, 3)

# Save to Excel
df.to_excel('fx_data.xlsx', index=False)

print("Saved to fx_data.xlsx")

Saved to fx_data.xlsx


**Train Process**

***Step 1：切訓練集 + 測試集***

In [3]:
# 自動依照比例切分訓練集與測試集
from sklearn.preprocessing import MinMaxScaler

# Step 1: 檢查資料長度
num_days = fx_data.shape[0]
print(f"共有 {num_days} 天的資料")

# Step 2: 切分比例（例如：80% 訓練）
train_ratio = 0.8
train_days = int(num_days * train_ratio)
print(f"訓練集長度: {train_days} 天")
print(f"測試集長度: {num_days - train_days} 天")

# Step 3: 正規化資料
scaler = MinMaxScaler()
scaler.fit(fx_data[:train_days])  # 用前 80% 做 scaling

fx_scaled = scaler.transform(fx_data)

# Step 4: 切出訓練集
window_size = 30
X_train, y_train = [], []
for i in range(train_days - window_size):
    X_train.append(fx_scaled[i:i+window_size])
    y_train.append(fx_scaled[i+window_size])
X_train = np.array(X_train)
y_train = np.array(y_train)


共有 3125 天的資料
訓練集長度: 2500 天
測試集長度: 625 天


**Step 2：建立並訓練模型**

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# 建立 GRU 模型
model = Sequential()
model.add(GRU(64, input_shape=(window_size, 3)))
model.add(Dense(3))  # 三個幣種輸出
model.compile(optimizer='adam', loss='mse')

# 訓練
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1)

# 儲存完整模型
model.save("fx_model_gru.h5")
print("模型已儲存為 fx_model_gru.h5")

# 接著儲存 scaler
import joblib
joblib.dump(scaler, "scaler.pkl")
print("Scaler 已儲存為 scaler.pkl")


Epoch 1/30


c:\Users\HP\anaconda3\envs\mll\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0537
Epoch 2/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0012
Epoch 3/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.9905e-04
Epoch 4/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.5005e-04
Epoch 5/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.3319e-04
Epoch 6/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.8135e-04
Epoch 7/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.4898e-04
Epoch 8/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.4001e-04
Epoch 9/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3949e-04
Epoch 10/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3094e-04
Epoch 11/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1400e-04
Epoch 12/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3136e-04
Epoch 13/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0767e-04
Epoch 14/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0520e-04
Epoch 15/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/st

模型已儲存為 fx_model_gru.h5
Scaler 已儲存為 scaler.pkl


***Step 3：開始做 Rolling 預測 + 驗證***

In [5]:
# 用訓練完的模型，從 day=1000 到 day=1089 做逐日預測
preds = []
true_vals = []

for i in range(train_days, len(fx_scaled) - 1):
    input_seq = fx_scaled[i - window_size:i]  # 拿前30天做預測
    input_seq = input_seq.reshape(1, window_size, 3)

    pred_scaled = model.predict(input_seq)[0]  # shape: (3,)
    true_scaled = fx_scaled[i]  # 真實第 i 天的資料

    # 還原成實際價格
    pred_real = scaler.inverse_transform([pred_scaled])[0]
    true_real = scaler.inverse_transform([true_scaled])[0]

    preds.append(pred_real)
    true_vals.append(true_real)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

***Step 4：計算誤差指標（RMSE、MAE)***

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

preds = np.array(preds)
true_vals = np.array(true_vals)

rmse = np.sqrt(mean_squared_error(true_vals, preds))
mae = mean_absolute_error(true_vals, preds)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")


RMSE: 0.6486
MAE: 0.3063


In [7]:
import matplotlib.pyplot as plt
import os

currency_names = ['USDJPY', 'EURUSD', 'GBPUSD']

# 建立 results 資料夾（若尚未存在）
save_dir = 'results'
os.makedirs(save_dir, exist_ok=True)

def get_unique_filename(base_name, ext='png'):
    """回傳不重複的檔案名稱（在 results 資料夾內）"""
    counter = 1
    filename = os.path.join(save_dir, f"{base_name}.{ext}")
    while os.path.exists(filename):
        filename = os.path.join(save_dir, f"{base_name}_{counter}.{ext}")
        counter += 1
    return filename

for i in range(3):
    plt.figure(figsize=(20, 4))
    plt.plot([t[i] for t in true_vals], label='True')
    plt.plot([p[i] for p in preds], label='Predicted')
    plt.title(f"Rolling Prediction - {currency_names[i]}")
    plt.xlabel("Days")
    plt.ylabel("Exchange Rate")
    plt.legend()
    plt.grid(True)

    filename = get_unique_filename(f"rolling_prediction_{currency_names[i]}")
    plt.savefig(filename)
    plt.close()
    print(f"Saved: {filename}")


Saved: results\rolling_prediction_USDJPY.png
Saved: results\rolling_prediction_EURUSD.png
Saved: results\rolling_prediction_GBPUSD.png


In [8]:
###################################### after training model #######################################
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

class FXTrading:
    def __init__(self, fx_rates, real_fx_rates):
        """
        fx_rates: shape (3, N) → 初始預測匯率，可用 real_fx_rates 的最後一段
        real_fx_rates: shape (3, N) → 所有實際匯率資料
        """
        self.Pre_fx_rates = fx_rates
        self.real_fx_rates = real_fx_rates
        self.day = 0
        self.start = len(fx_rates[0]) - window_size -1  # 模擬起始點

        self.initial_capital = np.array([1000, 1000, 1000], dtype=float)
        self.capital = np.array([1000, 1000, 1000], dtype=float)
        self.available_margin = self.capital

        self.leverage = np.array([5, 5, 5])
        self.position_size = np.array([0, 0, 0], dtype=float)
        self.position_value = np.array([0, 0, 0], dtype=float)
        self.floating_pnl = np.array([0, 0, 0], dtype=float)

        self.now_price = np.array([
            real_fx_rates[0][self.start],
            real_fx_rates[1][self.start],
            real_fx_rates[2][self.start]
        ], dtype=float)

        self.entry_price = np.array([0, 0, 0], dtype=float)
        self.margin = 10

        # ❶ 載入模型與 scaler
        self.model = load_model("fx_model_gru.h5", compile=False)
        self.scaler = joblib.load("scaler.pkl")
        self.window_size = 30  # 必須和訓練時一致

    def check_liquidation(self, cap_num, maintenance_margin_ratio_threshold=0.3):
        """
        Check if the position should be liquidated (trigger forced liquidation).

        Args:
            cap_num (int): Index of the currency (0, 1, 2)
            maintenance_margin_ratio_threshold (float): Liquidation threshold (default 30%)

        Returns:
            bool: True if should liquidate, False otherwise
        """
        equity = self.capital[cap_num] + self.floating_pnl[cap_num]

        if equity / (self.margin * abs(self.position_size[cap_num])) < maintenance_margin_ratio_threshold:
            return True

        return False

    def close_position(self, cap_num, close_price):
        """
        Close the position and calculate realized PnL.

        Args:
            cap_num (int): Index of the currency
            close_price (float): Current market price

        Returns:
            float: Realized PnL
        """
        return (close_price - self.entry_price[cap_num]) * self.position_size[cap_num] * self.margin * self.leverage[cap_num] / close_price

    def predict_fx_rate(self, data=None):
        """
        使用 GRU 模型預測下一日三種匯率，並更新 self.Pre_fx_rates。
        """
        # ❷ 取得最近 window_size 天的實際匯率（格式 shape: (30, 3)）
        recent_days = []
        for i in range(self.start + self.day - self.window_size, self.start + self.day):
            recent_days.append([
                self.real_fx_rates[0][i],
                self.real_fx_rates[1][i],
                self.real_fx_rates[2][i]
            ])
        recent_days = np.array(recent_days)  # shape: (30, 3)

        # ❸ 正規化
        scaled_input = self.scaler.transform(recent_days)
        scaled_input = scaled_input.reshape(1, self.window_size, 3)  # shape: (1, 30, 3)

        # ❹ 預測並還原
        scaled_pred = self.model.predict(scaled_input)[0]  # shape: (3,)
        real_pred = self.scaler.inverse_transform([scaled_pred])[0]  # shape: (3,)

        # ❺ 更新 self.Pre_fx_rates
        self.Pre_fx_rates = np.concatenate([self.Pre_fx_rates, real_pred.reshape(3, 1)], axis=1)

    def open_position(self, cap_num, any): #Needs to be designed manually (y)
        """
        Decide how to open a position based on predicted vs. current price.

        Returns:
            (int, float): (action, number of lots)
                action: 0 = LONG, 1 = SHORT, 2 = HOLD
                buy_num: number of lots
        """
        predicted_price = self.Pre_fx_rates[cap_num][self.start + self.day]
        current_price = self.now_price[cap_num]

        if predicted_price > current_price:
            return 0, 10  # LONG
        elif predicted_price < current_price:
            return 1, 10  # SHORT
        else:
            return 2, 0   # HOLD

    def decide_action(self, any): #Needs to be designed manually (y)
        """
        Decide what to do with an existing position.

        Returns:
            (int, float): (action, number of lots)
                action: 0 = ADD, 1 = CLOSE, 2 = HOLD
        """
        actions = []

        for cap_num in range(3):
            predicted_price = self.Pre_fx_rates[cap_num][self.start + self.day]
            current_price = self.now_price[cap_num]
            pos = self.position_size[cap_num]
            pnl = self.floating_pnl[cap_num]

            # 若賺超過 100 或賠超過 -100 就平倉
            if pnl > 100 or pnl < -100:
                actions.append((1, abs(pos)))  # CLOSE 全部
                continue

            # 若預測方向與持倉方向相反 → 平倉
            if (pos > 0 and predicted_price < current_price) or (pos < 0 and predicted_price > current_price):
                actions.append((1, abs(pos)))  # CLOSE
                continue

            # 若預測與方向一致，可選擇加碼
            actions.append((2, 0))  # HOLD

        return actions[self.day % 3]  # 每次 call 只會用一個 cap_num，所以這樣 round-robin 給個值

    def update_entry_price(self, cap_num, add_price, old_position, add_position):
        """
        Update the average entry price after adding position.

        Args:
            cap_num (int): Index of the currency
            add_price (float): Price of new added position
            old_position (float): Previous position size
            add_position (float): New added position size
        """
        old_value = abs(old_position) * self.margin * self.leverage[cap_num]
        add_value = abs(add_position) * self.margin * self.leverage[cap_num]

        self.entry_price[cap_num] = (self.entry_price[cap_num] * old_value + add_price * add_value) / (old_value + add_value)

    def update(self):
        """
        Update environment for current day (prices, margins, PnL, position value).
        """
        self.now_price = np.array([
            self.real_fx_rates[0][self.start + self.day],
            self.real_fx_rates[1][self.start + self.day],
            self.real_fx_rates[2][self.start + self.day]
        ])

        self.available_margin = self.capital - abs(self.position_size) * self.margin
        self.position_value = abs(self.margin * self.position_size * self.leverage)
        self.floating_pnl = self.position_size * (self.now_price - self.entry_price) * self.leverage * self.margin / self.now_price

    def run_days(self, max_days=None):
        """
        Run simulation over multiple days.

        Args:
            max_days (int): Number of days to run
        """
        for day in range(max_days):
            self.day = day
            self.update()
            self.predict_fx_rate(None) #

            # Print current state
            print("Day ", day + 1)
            print(" ")

            for i, name in enumerate(["USD/JPY", "USD/EUR", "USD/GBP"]):
                print(name + ":")
                print("Pre_fx_rate: ", self.Pre_fx_rates[i][day + self.start], "real_fx_rates: ", self.now_price[i])
                print("Capital: ", self.capital[i], "available_margin: ", self.available_margin[i], "position_size: ", self.position_size[i], "leverage: ", self.leverage[i])
                print("floating_pnl: ", self.floating_pnl[i], "entry_price: ", self.entry_price[i], "position_value: ", self.position_value[i])
                print(" ")

            # Main trading loop for each currency
            for cap_num in range(3):
                # Check for liquidation
                if self.position_size[cap_num] != 0 and self.check_liquidation(cap_num):
                    print("Liquidation triggered!")
                    self.capital[cap_num] += self.close_position(cap_num, self.now_price[cap_num])
                    self.position_size[cap_num] = 0

                # If no position, try to open new position
                if self.position_size[cap_num] == 0 and self.capital[cap_num] > 0:
                    action, num = self.open_position(cap_num, None)
                    if action == 0 and num * self.margin <= self.available_margin[cap_num]:
                        self.position_size[cap_num] += num
                        self.entry_price[cap_num] = self.now_price[cap_num]
                    elif action == 1 and num * self.margin <= self.available_margin[cap_num]:
                        self.entry_price[cap_num] = self.now_price[cap_num]
                        self.position_size[cap_num] -= num
                else:
                    # If position exists, decide what to do
                    action, num = self.decide_action(None)

                    if action == 0:  # ADD position
                        self.update_entry_price(cap_num, self.now_price[cap_num], self.position_size[cap_num], num)
                        self.position_size[cap_num] += num
                    elif action == 1:  # CLOSE position
                        self.capital[cap_num] += self.close_position(cap_num, self.now_price[cap_num])
                        self.position_size[cap_num] = 0

        # Final update after run
        self.capital += self.position_size * (self.now_price - self.entry_price) * self.leverage * self.margin / self.now_price
        print("Final Results:")
        print("USD/JPY:  capital", self.capital[0])
        print("USD/EUR:  capital", self.capital[1])
        print("USD/GBP:  capital", self.capital[2])
        print("Rate of Return: ", sum(self.capital) / sum(self.initial_capital))


**Run the simulation**

In [10]:
# df1 = pd.read_excel('fx_data.xlsx')
# fx_data = df1.values  # or df.to_numpy()
# fx_data = fx_data.T  # (3, N)

df2 = pd.read_excel('fake_fx_data.xlsx')
real_fx_data = df2.values
real_fx_data = real_fx_data.T

fx_data = real_fx_data[:, -30:]

#fx_rates and real_fx_rates inputs need to be NumPy arrays of shape (3, N)
env = FXTrading(fx_rates = fx_data, real_fx_rates = real_fx_data)
env.run_days(max_days = 90)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Day  1
 
USD/JPY:
Pre_fx_rate:  149.58731336412208 real_fx_rates:  150.6560059
Capital:  1000.0 available_margin:  1000.0 position_size:  0.0 leverage:  5
floating_pnl:  0.0 entry_price:  0.0 position_value:  0.0
 
USD/EUR:
Pre_fx_rate:  0.9326252916890817 real_fx_rates:  0.922609985
Capital:  1000.0 available_margin:  1000.0 position_size:  0.0 leverage:  5
floating_pnl:  0.0 entry_price:  0.0 position_value:  0.0
 
USD/GBP:
Pre_fx_rate:  0.8102207638298857 real_fx_rates:  0.789699972
Capital:  1000.0 available_margin:  1000.0 position_size:  0.0 leverage:  5
floating_pnl:  0.0 entry_price:  0.0 position_value:  0.0
 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Day  2
 
USD/JPY:
Pre_fx_rate:  148.128006 real_fx_rates:  107.7080001831055
Capital:  1000.0 available_margin:  900.0 position_size:  -10.0 leverage:  5
floating_pnl:  199.37240336781912 entry_price:  150.6560059 position_value:  500.0
 
USD/EUR:
Pre_fx_rate:  0.91918999 real_fx_rates:  0.89939